<a href="https://colab.research.google.com/github/Harold-Rodriguez/MNIST-SVM-Classification/blob/main/Mini_projet_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform
from sklearn.metrics import mean_squared_error

In [2]:
mnist = fetch_openml('mnist_784', version=1, cache=True)

X = mnist['data']
y = mnist['target'].astype(np.uint8)

X_train = X[:60000]
y_train = y[:60000]
X_test = X[60000:]
y_test = y[60000:]

In [3]:
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


LinearSVC(random_state=42)

In [4]:
y_pred = lin_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.8348666666666666

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

In [6]:
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train_scaled, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


LinearSVC(random_state=42)

In [7]:
y_pred = lin_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9214

In [8]:
svm_clf = SVC(gamma="scale")
svm_clf.fit(X_train_scaled[:10000], y_train[:10000])

SVC()

In [9]:
y_pred = svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9455333333333333

In [10]:
param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, verbose=2, cv=3)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END ..C=10.649007698832055, gamma=0.0072115711349220206; total time=   0.3s
[CV] END ..C=10.649007698832055, gamma=0.0072115711349220206; total time=   0.3s
[CV] END ..C=10.649007698832055, gamma=0.0072115711349220206; total time=   0.3s
[CV] END .....C=4.065372484882821, gamma=0.01995086491478706; total time=   0.3s
[CV] END .....C=4.065372484882821, gamma=0.01995086491478706; total time=   0.3s
[CV] END .....C=4.065372484882821, gamma=0.01995086491478706; total time=   0.3s
[CV] END ....C=5.589750522311288, gamma=0.002669646986671338; total time=   0.2s
[CV] END ....C=5.589750522311288, gamma=0.002669646986671338; total time=   0.2s
[CV] END ....C=5.589750522311288, gamma=0.002669646986671338; total time=   0.2s
[CV] END ....C=1.3888129744135338, gamma=0.08923853079455421; total time=   0.3s
[CV] END ....C=1.3888129744135338, gamma=0.08923853079455421; total time=   0.3s
[CV] END ....C=1.3888129744135338, gamma=0.08923

RandomizedSearchCV(cv=3, estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f9758c62590>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f9759121850>},
                   verbose=2)

In [11]:
rnd_search_cv.best_estimator_

SVC(C=4.681984090721019, gamma=0.0012511447449283424)

In [12]:
rnd_search_cv.best_score_

0.8639927352502204

In [13]:
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

SVC(C=4.681984090721019, gamma=0.0012511447449283424)

In [14]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
np.sqrt(mse)

0.2437553418218084

In [15]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
np.sqrt(mse)

0.6736467917239716

In [16]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.9728